In [18]:
%reload_ext autoreload
%autoreload 2

In [19]:
import sys
sys.path.append('/workspace/deep_birth_death/src')

In [20]:
import pickle
import numpy as np
import pandas as pd
import random
from time import time

from tensorflow.keras import backend as K
from tensorflow.keras.layers import Input, GlobalAveragePooling1D, MaxPooling1D, Concatenate
from tensorflow.keras.layers import Dense, Conv1D, Dropout, BatchNormalization, LeakyReLU, ELU
from tensorflow.keras import Model
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model

In [21]:
def shuffle_three_arrays(a, b, c):
    combined = list(zip(a, b, c))
    random.shuffle(combined)
    a_permuted, b_permuted, c_permuted = zip(*combined)
    
    return np.array(a_permuted), np.array(b_permuted), np.array(c_permuted)

# Load data

In [22]:
n_tips = ['674', '489', '87']
pickle_base = '/workspace/deep_birth_death/deep_learning/pickles/old_sims/dataset_'
data = dict()
for i in n_tips:
    with open(pickle_base + i + "_10k.pkl", 'rb') as f:
        data[i] = pickle.load(f)

In [23]:
max_size = 674


x_train_total= {k: [] for k in ["BD", "HE", "ME", "SAT", "SR", "WW"]}
x_test_total= {k: [] for k in ["BD", "HE", "ME", "SAT", "SR", "WW"]}
y_train_total= {k: [] for k in ["BD", "HE", "ME", "SAT", "SR", "WW"]}
y_test_total= {k: [] for k in ["BD", "HE", "ME", "SAT", "SR", "WW"]}
resc_fact_train_total = {k: [] for k in ["BD", "HE", "ME", "SAT", "SR", "WW"]}
resc_fact_test_total = {k: [] for k in ["BD", "HE", "ME", "SAT", "SR", "WW"]}

for tip in data:

    data[tip]['X_train'] = np.pad(data[tip]['X_train'], 
                                 pad_width=((0,0), (0, max_size - data[tip]['X_train'].shape[1])),
                                 mode='constant', constant_values=0)
    data[tip]['X_test'] = np.pad(data[tip]['X_test'], 
                                pad_width=((0,0), (0, max_size - data[tip]['X_test'].shape[1])),
                                mode='constant', constant_values=0)

    for label in np.unique(data[tip]['div_info_train']):
        div_scenario = label.split('/')[1].split('_')[0]

        x_train = data[tip]['X_train'][data[tip]['div_info_train'] == label]
        x_test = data[tip]['X_test'][data[tip]['div_info_test'] == label]

        y_train = data[tip]['y_reg_norm_train'][data[tip]['div_info_train'] == label]
        y_test = data[tip]['y_reg_test'][data[tip]['div_info_test'] == label]
        
        resc_fact_train = data[tip]['resc_factor_train'][data[tip]['div_info_train'] == label]
        resc_fact_test = data[tip]['resc_factor_test'][data[tip]['div_info_test'] == label]

        if div_scenario in x_train_total:
            x_train_total[div_scenario].extend(x_train)
            x_test_total[div_scenario].extend(x_test)
            y_train_total[div_scenario].extend(y_train)
            y_test_total[div_scenario].extend(y_test)
            resc_fact_train_total[div_scenario].extend(resc_fact_train)
            resc_fact_test_total[div_scenario].extend(resc_fact_test)

In [24]:
for k in x_train_total:
    x_train_total[k] = np.array(x_train_total[k])
    x_test_total[k] = np.array(x_test_total[k])
    y_train_total[k] = np.array(y_train_total[k])
    y_test_total[k] = np.array(y_test_total[k])
    resc_fact_train_total[k] = np.array(resc_fact_train_total[k])
    resc_fact_test_total[k] = np.array(resc_fact_test_total[k])

    x_train_total[k], y_train_total[k], resc_fact_train_total[k] = shuffle_three_arrays(x_train_total[k], y_train_total[k], resc_fact_train_total[k])
    x_test_total[k], y_test_total[k], resc_fact_test_total[k] = shuffle_three_arrays(x_test_total[k], y_test_total[k], resc_fact_test_total[k])
                                                          
    print(k)
    print(x_train_total[k].shape)
    print(y_train_total[k].shape)  
    print(resc_fact_train_total[k].shape)
    print(x_test_total[k].shape )
    print(y_test_total[k].shape )
    print(resc_fact_test_total[k].shape)

BD
(26904, 674)
(26904, 2)
(26904,)
(3096, 674)
(3096, 2)
(3096,)
HE
(26994, 674)
(26994, 2)
(26994,)
(3006, 674)
(3006, 2)
(3006,)
ME
(27056, 674)
(27056, 4)
(27056,)
(2944, 674)
(2944, 4)
(2944,)
SAT
(27036, 674)
(27036, 1)
(27036,)
(2964, 674)
(2964, 1)
(2964,)
SR
(26972, 674)
(26972, 5)
(26972,)
(3028, 674)
(3028, 5)
(3028,)
WW
(27038, 674)
(27038, 5)
(27038,)
(2962, 674)
(2962, 5)
(2962,)


# Train the regression model

In [25]:
def create_nn(n_out, div_scenario = None):
    input_data = Input(shape=(674, 1))


    x = Conv1D(16, kernel_size=3, padding='same')(input_data)
    x = ELU()(x)
    x = Conv1D(16, kernel_size=3, padding='same')(x)
    x = ELU()(x)
    x = MaxPooling1D(pool_size=2)(x)
    
    x = Conv1D(32, kernel_size=3, padding='same')(x)
    x = ELU()(x)
    x = Conv1D(32, kernel_size=3, padding='same')(x)
    x = ELU()(x)
    x = MaxPooling1D(pool_size=2)(x)
    
    x = Conv1D(64, kernel_size=3, padding='same')(x)
    x = ELU()(x)
    x = Conv1D(64, kernel_size=3, padding='same')(x)
    x = ELU()(x)
    x = MaxPooling1D(pool_size=2)(x)
    
    x = Conv1D(128, kernel_size=3, padding='same')(x)
    x = ELU()(x)
    x = Dropout(0.3)(x)

    x = GlobalAveragePooling1D()(x)

    x = Dense(32)(x)
    x = ELU()(x)
    x = Dropout(0.3)(x)
    
    x = Dense(n_out)(x)


    if div_scenario != "SAT":
        out_list = []
        for i in range(n_out):
            y = Dense(32)(x)
            y = ELU()(y)
            y = Dropout(0.3)(y)
            y = Dense(1)(y)
            y = LeakyReLU(alpha=10)(y)
            out_list.append(y)
        output_class = Concatenate()(out_list)
        
    elif div_scenario == "WW":
        out_list = []
        for i in range(n_out):
            y = Dense(32)(x)
            y = ELU()(y)
            y = Dropout(0.3)(y)
            y = Dense(1)(y)
            y = Linear(y)
            out_list.append(y)
        output_class = Concatenate()(out_list)                
        
    else:
        x = Dense(32)(x)
        x = ELU()(x)
        x = Dropout(0.3)(x)
        x = Dense(1)(x)
        output_class = LeakyReLU(alpha=10)(x)
            
    return Model(input_data, output_class)

In [ ]:
out_folder_path = "/workspace/deep_birth_death/deep_learning/models/testing_total_reg/"
callback = EarlyStopping(monitor='val_loss', patience=100, restore_best_weights=True)

for div_scenario in ["SAT" , "SR", "WW"]:
    
    print("\nRegression training: ", div_scenario)
    
    nn_model = create_nn(len(y_train_total[div_scenario][0]), div_scenario=div_scenario)
    nn_model.compile(loss="mae", optimizer=Adam(learning_rate=0.001), metrics=['mse'])    
    
    start = time()                     
    history = nn_model.fit(x_train_total[div_scenario],
                           y_train_total[div_scenario],
                           batch_size=128, epochs=1000, validation_split=0.1,
                           callbacks=[callback], verbose=1)                     
    elapsed_time = time()-start  
                         
    print('Elapsed time', elapsed_time)
              

    #Save the results
    save_path = out_folder_path + div_scenario + "_TOTAL_regression_" 
                         
    nn_model.save(save_path + "model.keras") 
    with open(save_path + "history.pkl", 'wb') as f:
        pickle.dump(history.history, f)
    with open(save_path + "model_data.pkl", 'wb') as f:
            pickle.dump([nn_model.count_params(), elapsed_time], f)
                    


Regression training:  SAT
Epoch 1/1000
191/191 [==============================] - 9s 42ms/step - loss: 1.1012 - mse: 2.4267 - val_loss: 0.4279 - val_mse: 0.3107
Epoch 2/1000
120/191 [=================>............] - ETA: 2s - loss: 0.6509 - mse: 0.6917

# Models evaluation

In [15]:
total_regression_results = dict()

for div_scenario in x_test_total:
    total_regression_results[div_scenario] = dict()
    
    # Load regression model 
    model_path = out_folder_path + div_scenario + "_TOTAL_regression_"  
    nn_model = load_model(model_path + 'model.keras')
    
    y_pred = nn_model.predict(x_test_total[div_scenario])

    y_test = y_test_total[div_scenario]     
    
    # Rescale back the values
    if div_scenario == "BD" or div_scenario == "HE" or div_scenario == "SAT":
                    
        y_pred[:, 0] = y_pred[:, 0] / resc_fact_test_total[div_scenario]
    
    elif div_scenario == "ME":

        y_pred[:, 0] = y_pred[:, 0] / resc_fact_test_total[div_scenario]
        y_pred[:, 2] = y_pred[:, 2] / resc_fact_test_total[div_scenario]
  
    else:
                    
        y_pred[:, 0] = y_pred[:, 0] / resc_fact_test_total[div_scenario]
        y_pred[:, 1] = y_pred[:, 1] / resc_fact_test_total[div_scenario]
        y_pred[:, 4] = y_pred[:, 4] / resc_fact_test_total[div_scenario]

    error = abs(y_pred-y_test)
    r_error = abs((y_pred - y_test)/ y_test)
    
    total_regression_results[div_scenario]['MAE'] = np.mean(error, axis=0)
    total_regression_results[div_scenario]['MRE'] = np.mean(r_error, axis=0)

OSError: SavedModel file does not exist at: /workspace/deep_birth_death/deep_learning/models/testing_total_reg/SR_TOTAL_regression_model.keras/{saved_model.pbtxt|saved_model.pb}

In [16]:
total_regression_results

{'BD': {'MAE': array([0.16890474, 0.08685253]),
  'MRE': array([0.08498494, 0.88742897])},
 'HE': {'MAE': array([0.27084062, 0.02065759]),
  'MRE': array([0.13442117, 0.02432648])},
 'ME': {'MAE': array([0.03656115, 0.08245859, 4.74847935, 0.04930744]),
  'MRE': array([0.14024427, 0.17142284, 0.68083257, 0.28396805])},
 'SAT': {'MAE': array([0.10877592]), 'MRE': array([0.05593215])},
 'SR': {}}

In [17]:
reg_values = {
    'BD': ['r', 'a'],
    'HE': ['r', 'a'],
    'ME': ['r', 'a', 'time', 'frac'],
    'SR': ['r0', 'r1', 'a0', 'a1', 'time'],
    'WW': ['r0', 'r1', 'a0', 'a1', 'time'],
    'SAT': ['lambda 0'],
}
    
for div_scenario in x_test_total:   
    param_names = reg_values[div_scenario]
    mae_vals = total_regression_results[div_scenario]['MAE']
    mre_vals = total_regression_results[div_scenario]['MRE']
    
    df = pd.DataFrame({
            "param": param_names,
            "MAE": mae_vals,
            "MRE": mre_vals
        })
    
    print("MAE and MRE for :" + div_scenario + " diversification scenario")
    print(df)

MAE and MRE for :BD diversification scenario
  param       MAE       MRE
0     r  0.168905  0.084985
1     a  0.086853  0.887429
MAE and MRE for :HE diversification scenario
  param       MAE       MRE
0     r  0.270841  0.134421
1     a  0.020658  0.024326
MAE and MRE for :ME diversification scenario
  param       MAE       MRE
0     r  0.036561  0.140244
1     a  0.082459  0.171423
2  time  4.748479  0.680833
3  frac  0.049307  0.283968
MAE and MRE for :SAT diversification scenario
      param       MAE       MRE
0  lambda 0  0.108776  0.055932


KeyError: 'MAE'